# Dimensão Tempo

Criação e carga da tabela de dimensão de tempo (calendário).

In [ ]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {fqtn(SCHEMA_GOLD, 'dim_tempo')} (
    date_key INT,
    full_date DATE,
    year INT,
    quarter INT,
    month INT,
    day_of_week INT,
    day_name STRING,
    month_name STRING,
    is_weekend BOOLEAN
) USING DELTA
""")

# Gerar dados de 2023 a 2025 se a tabela estiver vazia
if spark.read.table(fqtn(SCHEMA_GOLD, 'dim_tempo')).count() == 0:
    spark.sql(f"""
    WITH dates AS (
      SELECT date_add('2023-01-01', pos) as d
      FROM (SELECT posexplode(sequence(0, 1095)) as (pos, val))
    )
    INSERT INTO {fqtn(SCHEMA_GOLD, 'dim_tempo')}
    SELECT 
      year(d) * 10000 + month(d) * 100 + day(d) as date_key,
      d as full_date,
      year(d),
      quarter(d),
      month(d),
      dayofweek(d),
      date_format(d, 'EEEE') as day_name,
      date_format(d, 'MMMM') as month_name,
      dayofweek(d) IN (1, 7) as is_weekend
    FROM dates
    """)